In [76]:

import numpy as np
import matplotlib.pylab as plt
import time
from scipy import optimize
from scipy import integrate
# from numba import jit
import pandas as pd

In [77]:
Dim = 3
x = np.random.normal(0,1,(1000,Dim))
x[:,1] += 2.
x[:,2] = 2*x[:,2]

print(x)

[[-0.2087653   2.00385716  1.15402389]
 [ 1.37588506  1.96633002 -1.03121399]
 [-0.26382414  2.21233532 -1.01144043]
 ...
 [ 0.36401535  2.99659866 -0.47386784]
 [-0.41228805  2.21051468  2.06485551]
 [ 0.76262349  4.13518419  0.09294566]]


In [78]:
Library = np.ones((x.shape[0],1))
print(Library.shape)


for i in range(Dim): # linear
    Library = np.concatenate((Library,x[:,i].reshape(-1,1)),
                             axis = 1)
    
for i in range(Dim): # quadratic
    for j in range(i,Dim):
        Library = np.concatenate((Library,(x[:,i]*x[:,j]).reshape(-1,1)),
                             axis = 1)    

(1000, 1)


In [79]:
def D1(x):
    return(x[:,0] + 2*x[:,0]**2. + 0.01 * x[:,1]- 0.5*x[:,2]**2.)

D1(x[0,:].reshape(1,-1))

array([-0.7674464])

In [80]:
D_on_x = D1(x)
print(D_on_x.shape, type(D_on_x))
print(D_on_x)

(1000,) <class 'numpy.ndarray'>
[-7.67446400e-01  4.64996660e+00 -6.14000309e-01  1.02010636e+00
  4.10625998e+00 -2.58318178e+00 -3.04247862e+00 -3.34459767e+00
  1.27879688e+01 -3.10758641e+00 -1.43108146e+00  2.66868878e-02
 -6.60575508e+00  4.87400351e+00 -2.23314220e+00  3.78660964e-01
  1.92169713e+00 -4.02942864e-01 -2.61331460e+00  7.68953785e-01
  1.54385311e+00  3.31567110e-01  2.46744047e+00 -2.86098529e+00
 -2.42211383e+00 -5.75629713e-01 -7.87552370e-01 -1.65354986e+00
  5.00910433e+00 -1.02365078e+00 -1.63611628e-02  3.00128596e+00
 -6.06628189e+00 -1.61564254e+00 -3.64923660e+00  1.45295521e+00
 -3.33325790e+00  1.21085864e+00 -2.77975108e-01 -1.96581842e+00
 -1.51985318e-01 -3.05720716e+00  9.04744419e-01 -1.00889360e+00
 -7.05354163e+00 -4.29181282e-01 -1.72699912e-01  3.58798305e+00
  3.12430559e+00 -4.14569138e+00 -7.68003232e-01 -3.60461520e-01
 -3.27474119e+00 -4.01446253e+00 -1.17252621e+00 -2.81477193e+00
  7.36037941e-01  7.11089865e+00  1.09861168e+01  1.066886

### Ranges


In [81]:
x[0,:]

array([-0.2087653 ,  2.00385716,  1.15402389])

In [82]:
Ranges = np.empty((Dim,2)) # minimum and Maximum
for i in range(Dim):
    Ranges[i,:] = [min(x[:,i]), max(x[:,i])]
print(Ranges)

[[-3.52717724  2.87377689]
 [-0.77737524  4.80601491]
 [-7.56831277  6.5662825 ]]


### Simple Integral Approach

In [105]:
def Marginalise_over_One(x,i,f, Range_i, nx=100):
    # input: x0, x1, ....; i is the xi for which f is integrated over
    #        nx: number of increments for integration
    # note that the value for x_i is irrelevant for the algorithm
    # output: marginalised value for function f with respect to xi    
    Support = np.linspace(Range_i[0], Range_i[1], nx)
    dxi = Support[1] - Support[0] # Increment for integral
    dimx = len(x)
    #print("Increment", dxi)

    IntegrationArea = np.empty((nx,dimx))
    for j in range(dimx):
        if j != i: # for all xj input: always constant values
            IntegrationArea[:,j] = np.ones(nx)*x[j]
        elif j == i:
            IntegrationArea[:,j] = Support
            
    f_val = f(IntegrationArea)
    #print(IntegrationArea.shape, f_val.shape)
    #print(np.mean(f_val))
    #return(f_val.sum()*dxi /(Range_i[1]- Range_i[0]) )
    return(np.mean(f_val))

In [106]:
# No difference because of the strictly linear dependence of D1 on x1

print(Marginalise_over_One(np.array([1.,1.,1.]),
                    i = 1, f = D1, Range_i = Ranges[1,:], nx=100))
print(Marginalise_over_One(np.array([1.,1.,1.]),
                    i = 1, f = D1, Range_i = Ranges[1,:] ,nx = 1000))
print(Marginalise_over_One(np.array([1.,1.,1.]),
                    i = 1, f = D1,Range_i = Ranges[1,:], nx = 100000))
print(Marginalise_over_One(np.array([1.,1.,1.]),
                    i = 1, f = D1,Range_i = Ranges[1,:], nx = 1000000))

2.5201431983668563
2.5201431983668563
2.5201431983668567
2.5201431983668563


In [108]:
# Nonlinearity: there is now a difference

print(Marginalise_over_One(np.array([1.,1.,1.]),
                    i = 0, f = D1, Range_i = Ranges[0,:], nx=100))
print(Marginalise_over_One(np.array([1.,1.,1.]),
                    i = 0, f = D1, Range_i = Ranges[0,:] ,nx = 1000))
print(Marginalise_over_One(np.array([1.,1.,1.]),
                    i = 0, f = D1, Range_i = Ranges[0,:] ,nx = 10000))

6.363421715780958
6.239139209649433
6.2268340110225475


In [96]:
def Integrand(x,i,f,Range_i, nx=100):
    return( (f(x.reshape(1,-1)) - Marginalise_over_One( x,i, f, Range_i, nx ))**2. )


In [98]:
print(Integrand(np.array([1.,1.,1.]),
                    i = 0, f = D1, Range_i = Ranges[0,:], nx=1000))

Increment 0.006407361493212971
(1000, 3) (1000,)
6.239139209649433
[13.95309807]


In [99]:

def One(x0,x1,x2):
    return(1.)

def integrand(x0,x1,x2,i, f, Range_i, nx):
    return(Integrand(np.array([x0,x1,x2]), i, f , Range_i , nx))

In [109]:
Volume, verror = integrate.nquad(One,[[-3,3], [-1,5], [-7,6]])
print(Volume, verror)
for i in range(Dim):
    
    result, error = integrate.nquad(integrand, [[-3,3], [-1,5], [-7,6]],
                               args = (i, D1, Ranges[i,:], 10000)) 
    print(i, result/Volume, error)

468.0 5.195843755245733e-12
0 32.31385099935867 1.6789772092295883e-10
1 0.00030002050577227087 1.55885967136252e-15
2 44.8400002599612 2.3298163533593645e-10


## Now with the 6D-Coupled-Lorenz

In [111]:
BIC= pd.read_csv("Lorenz_Score_BIC.csv", index_col = 0)
print(BIC)
print(BIC["BIC"]==min(BIC["BIC"]))
logic = BIC["BIC"]==min(BIC["BIC"])
print(np.arange(3)[logic])

           BIC  Threshold
0  1876.523805       0.05
1  1747.523895       0.55
2  7867.645786       1.05
0    False
1     True
2    False
Name: BIC, dtype: bool
[1]


In [112]:

filename = 'CoupledLorenz_dt0.01' 
x = np.load(filename+'.npy')
print(x.shape)

(5000, 6)


In [113]:
BestAlphas = np.loadtxt('CoupledLorenz_Alpha_Done.txt')[BIC["BIC"]==min(BIC["BIC"]),:]

In [114]:
RangeL = np.empty((6,2))
Dim = x.shape[1]    # NOW NEW DIMENSION!
for i in range(Dim):
    RangeL[i,:] = [min(x[:,i]), max(x[:,i])] 
print(RangeL)

[[-19.32628135  21.26101002]
 [-24.5107052   27.94186795]
 [  1.          52.2962196 ]
 [-22.45140621  20.64704473]
 [-28.70851393  24.51948398]
 [  3.74703334  53.52157254]]


In [115]:
# let's only regard up to second order    
def poly(x,sigma):
    x_vec=np.array([1,x[0],x[1],x[2],x[3],x[4],x[5],  #7
                   x[0]**2., x[1]**2., x[2]**2., x[3]**2., x[4]**2., x[5]**2., # 6 terms
                    x[0]*x[1], x[0]*x[2], x[0]*x[3], x[0]*x[4], x[0]*x[5], # 5 terms
                    x[1]*x[2], x[1]*x[3], x[1]*x[4], x[1]*x[5], # 4 terms
                    x[2]*x[3], x[2]*x[4], x[2]*x[5], # 3 terms
                    x[3]*x[4], x[3]*x[5], x[4]*x[5]]) # 3 terms
    
    return np.dot(sigma,x_vec) # Total: 28 terms

#@jit
def D1(x):
    sigma = BestAlphas.flatten()[1:] # without noise parameters
    sigma=sigma.reshape((Dim,-1))
    function=np.zeros((len(x),Dim))
    for i in range(Dim):
        function[:,i]=poly(x.T,sigma[i])
    return function

In [123]:
Sigma_Vec = BestAlphas.flatten()

def D1_j(x0,x1,x2,x3,x4,x5,j):
    # returns the j'th component of D1
    x_arr = np.array([x0,x1,x2,x3,x4,x5])
    return(D1(x_arr)[0,j])

for j in range(6):
    print(D1_j(x[0,0], x[0,1],x[0,2], x[0,3],x[0,4],x[0,5],j))
    
    
def D1_0(x):
    return(D1(x)[0,0])
    
def D1_1(x):
    return(D1(x)[0,1])
    
def D1_2(x):
    return(D1(x)[0,2])
    
def D1_3(x):
    return(D1(x)[0,3])
    
def D1_4(x):
    return(D1(x)[0,4])
    
def D1_5(x):
    return(D1(x)[0,5])


D1_0(x[0,:])

0.9161577529163942
26.622084909093488
0.1684962347598935
15.586333694423445
161.0531733434817
76.04350914923002


0.9161577529163942

In [117]:
for j in range(Dim):
    print(j, Marginalise_over_One(np.array([1.,2.,1.,1.,1.,1.]),
                    i = j, f = D1_0, Range_i = RangeL[j,:], nx=100))

0 204.60580040364079
1 -250.00982860886214
2 10.752263197641675
3 10.752263197641675
4 10.752263197641675
5 10.752263197641675


/home/users/t_wand01/.local/lib/python3.7/site-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [126]:
for j in range(Dim):
    print(j, Marginalise_over_One(np.array([1.,2.,1.,1.,1.,1.]),
                    i = j, f = D1_3, Range_i = RangeL[j,:], nx=100))

0 6.113916317059198
1 6.113916317059198
2 6.113916317059198
3 231.8780609787135
4 -311.15466405335536
5 6.113916317059198


/home/users/t_wand01/.local/lib/python3.7/site-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


### Normalisation of the phase space volume

In [27]:
def One6(x0,x1,x2,x3,x4,x5):
    return(1)

In [28]:
Volume, verror = integrate.nquad(One6,RangeL)
print(Volume, verror)


12469563969.440876 0.00013843997025909523


In [29]:
print(RangeL)
V = 1
for i in range(6):
    V = V*(RangeL[i,1]-RangeL[i,0])
print(V)

[[-19.32628135  21.26101002]
 [-24.5107052   27.94186795]
 [  1.          52.2962196 ]
 [-22.45140621  20.64704473]
 [-28.70851393  24.51948398]
 [  3.74703334  53.52157254]]
12469563969.440868


In [30]:
Marginalise_over_One

<function __main__.Marginalise_over_One(x, i, f, Range_i, nx=100)>

In [31]:
print(Marginalise_over_One(np.array([1.,1.,1.,1.,1.,1.]),
                    i = 1, f = D1, Range_i = Ranges[1,:], nx=100))

68.56006169821465


/home/users/t_wand01/.local/lib/python3.7/site-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [136]:
def Integrand(x,i,f,Range_i, nx=100):
    return( (f(x.reshape(1,-1)) - Marginalise_over_One( x,i, f, Range_i, nx ))**2. )

print(D1_0(np.ones(6).reshape(1,-1)))
print(Marginalise_over_One(np.array([1.,1.,1.,1.,1.,1.]),
                    i = 0, f = D1_0, Range_i = RangeL[0,:], nx=1000))

print(Integrand(np.array([1.,1.,1.,1.,1.,1.]),
                    i = 0, f = D1_0, Range_i = RangeL[0,:], nx=1000))

def integrand6(x0,x1,x2,x3,x4,x5,i, f, Range_i, nx):
    return(Integrand(np.array([x0,x1,x2,x3,x4,x5]), i, f , Range_i , nx))

print(integrand6(1,1,1,1,1,1,i=0, f=D1_0, Range_i = RangeL[0,:],nx=1000))

0.9161577529163942
194.76969495891552
37579.19388727768
37579.19388727768


/home/users/t_wand01/.local/lib/python3.7/site-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [137]:
for i in range(Dim):
    
    result, error = integrate.nquad(integrand6, RangeL,
                               args = (i, D1_0, Ranges[i,:], 100)) 
    print(i, result/Volume, error)

/home/users/t_wand01/.local/lib/python3.7/site-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


KeyboardInterrupt: 